# 日本のCOVID19感染者数分析(情報提供:NHK)

In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")

In [3]:
pref["日付"] = pd.to_datetime(pref["日付"] )

In [4]:
latest_date_str = pref["日付"].max().strftime("%Y/%m/%d")

In [5]:
all_jp = pref.copy().groupby(["日付"], as_index=False).sum()
del all_jp["都道府県コード"]
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]

In [6]:
all_jp["weekday"] = all_jp["日付"].dt.weekday
weekdays = pd.DataFrame({"曜日":["月","火","水","木","金","土","日"]})
all_jp = pd.merge(all_jp, weekdays, left_on="weekday", right_index=True, how="left")

## 移動平均

In [7]:
all_jp["感染者数移動平均7"] = all_jp["各地の感染者数_1日ごとの発表数"].rolling(7).mean()
all_jp["死者数移動平均7"] = all_jp["各地の死者数_1日ごとの発表数"].rolling(7).mean()
all_jp.tail()

,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,各地の直近1週間の人口10万人あたりの感染者数,weekday,曜日,感染者数移動平均7,死者数移動平均7
858,2022-05-23,18395,31,9350.32,0,月,34481.000000,35.571429
859,2022-05-24,32283,42,9165.10,1,火,33836.428571,36.000000
860,2022-05-25,35110,43,8864.45,2,水,32842.571429,35.000000
861,2022-05-26,30924,42,8512.04,3,木,31605.857143,35.857143
862,2022-05-27,27461,38,8118.73,4,金,30194.285714,35.000000


## 増減速度

In [8]:
all_jp["感染者数移動平均7増減"] = all_jp["感染者数移動平均7"].diff()
all_jp["死者数移動平均7増減"] = all_jp["死者数移動平均7"].diff()
all_jp.tail(7)

,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,各地の直近1週間の人口10万人あたりの感染者数,weekday,曜日,感染者数移動平均7,死者数移動平均7,感染者数移動平均7増減,死者数移動平均7増減
856,2022-05-21,35855,31,9671.64,5,土,35457.428571,35.857143,-496.428571,0.142857
857,2022-05-22,31332,18,9495.57,6,日,34949.285714,34.857143,-508.142857,-1.000000
858,2022-05-23,18395,31,9350.32,0,月,34481.000000,35.571429,-468.285714,0.714286
859,2022-05-24,32283,42,9165.10,1,火,33836.428571,36.000000,-644.571429,0.428571
860,2022-05-25,35110,43,8864.45,2,水,32842.571429,35.000000,-993.857143,-1.000000
861,2022-05-26,30924,42,8512.04,3,木,31605.857143,35.857143,-1236.714286,0.857143
862,2022-05-27,27461,38,8118.73,4,金,30194.285714,35.000000,-1411.571429,-0.857143


### 感染者数前週同曜日比

In [9]:
import datetime
all_jp["新規感染者数先週比"] = all_jp["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)
all_jp["死者数先週比"] = all_jp["各地の死者数_1日ごとの発表数"].pct_change(periods=7)
all_jp["新規感染者数先週差"] = all_jp["各地の感染者数_1日ごとの発表数"].diff(periods=7)
all_jp["死者数先週差"] = all_jp["各地の死者数_1日ごとの発表数"].diff(periods=7)


## 都道府県別

In [10]:
pref["感染者数移動平均7"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).mean()

### 人口あたり

In [11]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)
pref = pd.merge(pref, pop, on="都道府県名", how="left")

In [12]:
pref["10万人あたり感染者数"] = pref["各地の感染者数_1日ごとの発表数"]/(pref["人口"]/100000)
pref["10万人あたり感染者数移動平均7"] = pref["10万人あたり感染者数"].rolling(7).mean()
pref["10万人あたり感染者数1週間合計"] = pref["10万人あたり感染者数"].rolling(7).sum()

### 感染者数前日比

In [13]:
import numpy as np
pref["感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
pref["感染者数前日差"] = pref["各地の感染者数_1日ごとの発表数"].diff()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"感染者数前日比"] = np.nan
pref["感染者数前日比移動平均7"] = pref["感染者数前日比"].rolling(7).mean()
pref["10万人あたり感染者数移動平均7前日差"] = pref["10万人あたり感染者数移動平均7"].diff()
pref["感染者数移動平均7前日差"] = pref["感染者数移動平均7"].diff()

### 感染者数前週同曜日比

In [14]:
import datetime
# 一番古い日付から一週間分NANを設定
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)
pref["新規感染者数先週差"] = pref["各地の感染者数_1日ごとの発表数"].diff(periods=7)
pref.loc[(pref["日付"] < pref["日付"].min()+datetime.timedelta(days=7)),"新規感染者数先週比"] = np.nan
pref.loc[(pref["日付"] < pref["日付"].min()+datetime.timedelta(days=7)),"新規感染者数先週差"] = np.nan


## 重症者数

In [15]:
severe = pd.read_csv("COVID-19-jp/severe_cases_daily.csv")
severe["Date"] = pd.to_datetime(severe["Date"])
#severe = severe[severe.Prefecture=="ALL"]
severe = severe[["Date","ALL"]].rename(columns={"ALL":"Severe cases"})

severe.tail()

,Date,Severe cases
743,2022-05-22,96
744,2022-05-23,100
745,2022-05-24,103
746,2022-05-25,95
747,2022-05-26,90


In [16]:
print(severe.count())
print(all_jp.count())

Date            748
Severe cases    748
dtype: int64
日付                         863
各地の感染者数_1日ごとの発表数           863
各地の死者数_1日ごとの発表数            863
各地の直近1週間の人口10万人あたりの感染者数    863
weekday                    863
曜日                         863
感染者数移動平均7                  857
死者数移動平均7                   857
感染者数移動平均7増減                856
死者数移動平均7増減                 856
新規感染者数先週比                  851
死者数先週比                     803
新規感染者数先週差                  856
死者数先週差                     856
dtype: int64


## 曜日別陽性者集計

In [17]:
all_jp_week_mean = all_jp.groupby(["weekday"],as_index=False).mean()[["各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数"]].sort_index()
all_jp_week_mean = pd.merge(all_jp_week_mean, weekdays, left_index=True, right_index=True, how="left")
all_jp_week_mean

,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,曜日
0,7016.642276,29.894309,月
1,9856.861789,42.495935,火
2,11480.008130,41.170732,水
3,11291.846774,38.919355,木
4,10925.685484,39.693548,金
5,10715.959350,32.300813,土
6,9703.593496,22.829268,日


In [18]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [19]:
fig=plt.figure(figsize=(9.5,8))
plt.subplot(2,1,1) # (rows, columns, panel number)
plt.title("国内の陽性者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の感染者数_1日ごとの発表数"],label="陽性者")
plt.bar(severe["Date"],severe["Severe cases"],label="重症者数")
plt.plot_date(all_jp["日付"],all_jp["感染者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.ylabel("人")
plt.grid()
plt.legend()

plt.subplot(2,1,2) # (rows, columns, panel number)
plt.title("国内の死者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の死者数_1日ごとの発表数"],label="死者")
plt.plot_date(all_jp["日付"],all_jp["死者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.grid()
plt.legend()

fig.tight_layout()


<IPython.core.display.Javascript object>

/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 20154 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 22269 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 20869 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 12398 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 38525 missing from current font.
  

In [20]:
fig=plt.figure(figsize=(9.5,8),tight_layout=True)
plt.subplot(2,1,1) # (rows, columns, panel number)
#fig, ax = plt.subplots(2,1,figsize=(9,8),tight_layout=True)
#wp_ax = ax[0]
plt.title("国内の曜日別陽性者数平均({}時点)".format(latest_date_str))
plt.bar(all_jp_week_mean["曜日"],all_jp_week_mean["各地の感染者数_1日ごとの発表数"])
#wd_ax = ax[1]
plt.subplot(2,1,2) # (rows, columns, panel number)
plt.title("国内の曜日別平均死者数平均({})時点)".format(latest_date_str))
plt.bar(all_jp_week_mean["曜日"],all_jp_week_mean["各地の死者数_1日ごとの発表数"])

<IPython.core.display.Javascript object>

<BarContainer object of 7 artists>

## 直近の推移

In [21]:
recent_date = pd.to_datetime(datetime.datetime.now()).round("D")- datetime.timedelta(days=90)
all_jp_recent = all_jp.loc[all_jp["日付"]>=recent_date]
all_jp_recent.tail(30)

,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,各地の直近1週間の人口10万人あたりの感染者数,weekday,曜日,感染者数移動平均7,死者数移動平均7,感染者数移動平均7増減,死者数移動平均7増減,新規感染者数先週比,死者数先週比,新規感染者数先週差,死者数先週差
833,2022-04-28,41684,39,9875.27,3,木,39770.857143,43.285714,-767.571429,-1.714286,-0.114181,-0.235294,-5373.0,-12.0
834,2022-04-29,36610,45,9674.80,4,金,38867.000000,42.428571,-903.857143,-0.857143,-0.147355,-0.117647,-6327.0,-6.0
835,2022-04-30,25099,14,9001.40,5,土,36181.857143,39.571429,-2685.142857,-2.857143,-0.428204,-0.588235,-18796.0,-20.0
836,2022-05-01,26880,34,8627.21,6,日,34524.000000,42.285714,-1657.857143,2.714286,-0.301546,1.266667,-11605.0,19.0
837,2022-05-02,19262,30,8468.82,0,月,33737.428571,41.000000,-786.571429,-1.285714,-0.222303,-0.230769,-5506.0,-9.0
838,2022-05-03,30428,55,8153.01,1,火,32308.571429,39.571429,-1428.857143,-1.428571,-0.247391,-0.153846,-10002.0,-10.0
839,2022-05-04,26417,20,7501.08,2,水,29482.857143,33.857143,-2825.714286,-5.714286,-0.428166,-0.666667,-19780.0,-40.0
840,2022-05-05,20710,18,6830.93,3,木,26486.571429,30.857143,-2996.285714,-3.000000,-0.503167,-0.538462,-20974.0,-21.0
841,2022-05-06,21535,37,6352.32,4,金,24333.000000,29.714286,-2153.571429,-1.142857,-0.411773,-0.177778,-15075.0,-8.0
842,2022-05-07,39234,26,7014.55,5,土,26352.285714,31.428571,2019.285714,1.714286,0.563170,0.857143,14135.0,12.0


In [22]:
severe_recent = severe.loc[severe["Date"]>=recent_date]
severe_recent.tail(30)

,Date,Severe cases
718,2022-04-27,174
719,2022-04-28,173
720,2022-04-29,171
721,2022-04-30,165
722,2022-05-01,164
723,2022-05-02,160
724,2022-05-03,174
725,2022-05-04,166
726,2022-05-05,170
727,2022-05-06,165


In [23]:
fig=plt.figure(figsize=(12,8))
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("国内の陽性者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の感染者数_1日ごとの発表数"],label="陽性者(infected)")
plt.bar(severe["Date"],severe["Severe cases"],label="重症者(severe)")
plt.plot_date(all_jp["日付"],all_jp["感染者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.ylabel("人(people)")
plt.grid()
plt.legend(fontsize=9)

plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("国内の死者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp["日付"],all_jp["各地の死者数_1日ごとの発表数"],label="死者(death)")
plt.plot_date(all_jp["日付"],all_jp["死者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x",rotation=70, labelsize=9)
plt.grid()
plt.legend(fontsize=9)

plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("直近の国内の陽性者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp_recent["日付"],all_jp_recent["各地の感染者数_1日ごとの発表数"],label="陽性者(infected)")
plt.bar(severe_recent["Date"],severe_recent["Severe cases"],label="重症者数(severe)")
plt.plot_date(all_jp_recent["日付"],all_jp_recent["感染者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
#plt.plot_date(all_jp_latest["日付"],all_jp_latest["直近１週間の新規感染者数合計"]/10,label="直近１週間の合計(10人)", fmt="-",lw=2.0, color="m")
plt.tick_params(axis="x",labelsize=9)
plt.ylabel("人(people)")
plt.grid()
latest_date = all_jp_recent["日付"].max()
latest_value = all_jp_recent.loc[all_jp_recent["日付"]==latest_date,"各地の感染者数_1日ごとの発表数"].iloc[0]
plt.text(latest_date,latest_value ,"{:,}".format(latest_value),fontsize=8,va="bottom")
#latest_value = severe_recent.loc[severe_recent["Date"]==latest_date,"Severe cases"].iloc[0]
#plt.text(latest_date,latest_value ,"{:,}".format(latest_value),fontsize=8,va="bottom",ha="left")
plt.legend(fontsize=9)

plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("直近の国内の死者数推移({}時点)".format(latest_date_str))
plt.bar(all_jp_recent["日付"],all_jp_recent["各地の死者数_1日ごとの発表数"],label="死者(death)")
plt.plot_date(all_jp_recent["日付"],all_jp_recent["死者数移動平均7"],label="7日間移動平均", fmt="-",lw=2.0, color="orange")
plt.tick_params(axis="x", labelsize=9)
plt.ylabel("人(people)")
plt.grid()
latest_value = all_jp_recent.loc[all_jp_recent["日付"]==latest_date,"各地の死者数_1日ごとの発表数"].iloc[0]
plt.text(latest_date,latest_value ,"{:,}".format(latest_value),fontsize=8,va="bottom",ha="center")
plt.legend(fontsize=9)

fig.text(1,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=8,ha="right")

fig.tight_layout()

<IPython.core.display.Javascript object>

/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 20154 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 22269 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 20869 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 12398 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 38525 missing from current font.
  

In [24]:
fig.savefig("out/covid-nhk-domestic-daily.png")

/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 20154 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 22269 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 20869 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 12398 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 38525 missing from current font.
  font.set

### 都道府県別

In [25]:
prefs = {
    "東京都": ("-",1.1,""),
#    "沖縄県": ("-",1.1,""),
    "埼玉県": ("-",1.1,""),
    "千葉県": ("-",1.1,""),
    "神奈川県": ("-",1.1,""),
#    "栃木県": ("-",1.1,""),
#    "茨城県": ("-",1.1,""),
#    "群馬県": ("-",1.1,""),
#    "静岡県": ("-",1.1,""),
    "京都府": ("-",1.1,""),
    "大阪府": ("-",1.1,""),
    "兵庫県": ("-",1.1,""),
#    "福岡県": ("-",1.1,""),
#    "北海道": ("--",1.0,"2021-05-25"),
#    "石川県": ("--",1.0,""),
#    "福島県": ("--",1.0,""),
#    "愛知県": ("--",1.0,"2021-05-25"),
#    "滋賀県": ("--",1.0,""),
#    "熊本県": ("--",1.0,""),
#    "広島県": ("--",1.0,"2021-04-15"),
#    "岐阜県": ("--",1.0,"2021-05-16"),
#    "三重県": ("--",1.0,"2021-05-27"),
#    "山梨県": (":",1.0,""),
#    "佐賀県": (":",1.5,""),
#    "奈良県": (":",1.5,""),
#    "山口県": (":",1.5,""),
#    "高知県": (":",1.5,""),
#    "青森県": (":",1.5,""),
#    "富山県": (":",1.5,""),
#    "岩手県": (":",1.5,""),
#    "島根県": (":",1.5,""),
}

In [26]:
pref_recent = pref.loc[pref["日付"]>=recent_date]

In [27]:
fig = plt.figure(figsize=(9.5,7))
plt.title("主要都道府県の１０万人当たり感染者数１週間合計の推移({}時点)".format(latest_date_str))
dates = pref_recent["日付"].unique()
for p in prefs:
    values = pref_recent.loc[pref_recent["都道府県名"]==p]["10万人あたり感染者数1週間合計"]
    plt.plot_date(dates,values,label=p, fmt=prefs[p][0], lw=prefs[p][1])
    if prefs[p][2] == "":
        text_date = dates.max()
    else:
        text_date = pd.to_datetime(prefs[p][2])
    text_y = pref_recent.loc[(pref_recent["都道府県名"]==p) & (pref_recent["日付"]==text_date)]["10万人あたり感染者数1週間合計"]
    plt.text(text_date,text_y,p,fontsize=8,va="center")
plt.plot([dates.min(),dates.max()],[15,15],"-",lw=1.5,color="black")
plt.text(dates.min(),15,"Stage3",ha="right",va="center",fontsize=9)
plt.plot([dates.min(),dates.max()],[25,25],"-",lw=1.5,color="black")
plt.text(dates.min(),25,"Stage4",ha="right",va="center",fontsize=9)
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("人数（人口１０万人あたり）")
fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

plt.tight_layout()

<IPython.core.display.Javascript object>

/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 20154 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 25968 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 65288 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 21475 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 65297 missing from current font.
  

In [28]:
fig.savefig("out/covid-nhk-domestic-major-pref-daily-recent.png")

/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 20154 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 25968 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 65288 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 21475 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:203: RuntimeWarning: Glyph 65297 missing from current font.
  font.set

In [29]:
fig = plt.figure(figsize=(10,10),tight_layout=True)
fig.text(0.5,1,"全都道府県の１０万人当たり感染者数１週間合計の推移({}時点)".format(latest_date_str),va="top",ha="center")
fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)
dates = pref_recent["日付"].unique()
all_prefs = pref_recent["都道府県名"].unique()
for i,p in enumerate(all_prefs):
    plt.subplot(7,7,i+1) # (rows, columns, panel number)
    plt.title(p,fontsize=8,y=0.7)
    values = pref_recent.loc[pref_recent["都道府県名"]==p]["10万人あたり感染者数1週間合計"]
    if p in prefs:
        fmt=prefs[p][0]
        lw=prefs[p][1]
    else:
        fmt=":"
        lw=0.5
#    plt.plot_date(dates,values,fmt=fmt, lw=lw)
    plt.plot_date(dates,values,fmt="-", lw=0.5)
    plt.tick_params(axis="x",labelsize=8,rotation=90)
    plt.tick_params(axis="y",labelsize=8)
    

"""
    if prefs[p][2] == "":
        text_date = dates.max()
    else:
        text_date = pd.to_datetime(prefs[p][2])
    text_y = pref_recent.loc[(pref_recent["都道府県名"]==p) & (pref_recent["日付"]==text_date)]["10万人あたり感染者数1週間合計"]
    plt.text(text_date,text_y,p,fontsize=8,va="center")
plt.plot([dates.min(),dates.max()],[15,15],"-",lw=1.5,color="black")
plt.text(dates.min(),15,"Stage3",ha="right",va="center",fontsize=9)
plt.plot([dates.min(),dates.max()],[25,25],"-",lw=1.5,color="black")
plt.text(dates.min(),25,"Stage4",ha="right",va="center",fontsize=9)
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("人数（人口１０万人あたり）")
"""


<IPython.core.display.Javascript object>

'\n    if prefs[p][2] == "":\n        text_date = dates.max()\n    else:\n        text_date = pd.to_datetime(prefs[p][2])\n    text_y = pref_recent.loc[(pref_recent["都道府県名"]==p) & (pref_recent["日付"]==text_date)]["10万人あたり感染者数1週間合計"]\n    plt.text(text_date,text_y,p,fontsize=8,va="center")\nplt.plot([dates.min(),dates.max()],[15,15],"-",lw=1.5,color="black")\nplt.text(dates.min(),15,"Stage3",ha="right",va="center",fontsize=9)\nplt.plot([dates.min(),dates.max()],[25,25],"-",lw=1.5,color="black")\nplt.text(dates.min(),25,"Stage4",ha="right",va="center",fontsize=9)\nplt.grid()\nplt.legend(bbox_to_anchor=(0, 1),loc=\'upper left\', fontsize=9)\nplt.ylabel("人数（人口１０万人あたり）")\n'

## 感染速度

In [30]:
fig = plt.figure(figsize=(9.5,10),tight_layout=True)
ax1 = plt.subplot(3,1,1) # (rows, columns, panel number)
plt.title("直近の国内の感染速度（７日間移動平均の増減）({}時点)".format(latest_date_str))
plt.plot_date(all_jp_recent["日付"],all_jp_recent["感染者数移動平均7増減"],label="陽性者",fmt="-")
plt.tick_params(axis="x",labelsize=9)
plt.ylabel("陽性者数増減（人）")
plt.grid()
plt.ylim(-1500,1500)
ax2 = plt.twinx()
ax2.plot_date(all_jp_recent["日付"],all_jp_recent["死者数移動平均7増減"],label="死者",fmt="-", color="orange")
plt.ylabel("死者数増減（人）")
plt.ylim(-25,25)
h1,l1 = ax1.get_legend_handles_labels()
h2,l2 = ax2.get_legend_handles_labels()
ax2.legend(h1+h2,l1+l2)

chart_data = pref_recent
dates = chart_data["日付"].unique()

ax1 = plt.subplot(3,1,2) # (rows, columns, panel number)
plt.title("首都圏１都３県の感染速度（１０万人当たり新規感染者数７日間移動平均の増減）({}時点)".format(latest_date_str))
target_p = ["東京都","神奈川県","埼玉県","千葉県"]
#stack_values = []
for p in target_p:
#    stack_values.append(chart_data.loc[chart_data["都道府県名"]==p]["10万人あたり感染者数移動平均7前日差"])
    values = chart_data.loc[chart_data["都道府県名"]==p]["10万人あたり感染者数移動平均7前日差"]
#    plt.fill_between(dates,values)
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
#plt.stackplot(dates, stack_values, labels=target_p)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("１０万人あたり新規感染者数増減")

ax1 = plt.subplot(3,1,3) # (rows, columns, panel number)
plt.title("関西圏２府３県の感染速度（１０万人当たり新規感染者数７日間移動平均の増減）({}時点)".format(latest_date_str))
target_p = ["大阪府","兵庫県","奈良県","京都府","和歌山県"]
for p in target_p:
    values = chart_data.loc[chart_data["都道府県名"]==p]["10万人あたり感染者数移動平均7前日差"]
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("１０万人あたり新規感染者数増減")


fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

<IPython.core.display.Javascript object>

Text(0, 0, '※新型コロナ関連の情報提供:NHK ／\u3000※人口の情報提供:e-Stat')

In [31]:
#fig.savefig("out/covid-nhk-domestic-velocity-daily.png")

## 感染速度2(前週同曜日比)

In [32]:
fig = plt.figure(figsize=(9.5,10),tight_layout=True)
ax1 = plt.subplot(3,1,1) # (rows, columns, panel number)
plt.title("直近の国内の感染速度（前週同曜日差の推移）({}時点)".format(latest_date_str))
plt.plot_date(all_jp_recent["日付"],all_jp_recent["新規感染者数先週差"],label="陽性者",fmt="-")
#plt.plot_date(all_jp["日付"],all_jp["新規感染者数先週差"],label="陽性者",fmt="-")
plt.tick_params(axis="x",labelsize=9)
plt.ylabel("陽性者数前週同曜日差(人)")
plt.grid()
plt.ylim(-35000,35000)
ax2 = plt.twinx()
ax2.plot_date(all_jp_recent["日付"],all_jp_recent["死者数先週差"],label="死者",fmt="-", color="orange")
plt.ylabel("死者数前週同曜日差(人)")
plt.ylim(-100,100)
plt.plot([dates.min(),dates.max()],[0,0],lw=1.1,color="black")
h1,l1 = ax1.get_legend_handles_labels()
h2,l2 = ax2.get_legend_handles_labels()
ax2.legend(h1+h2,l1+l2,bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)

chart_data = pref_recent
dates = chart_data["日付"].unique()

ax1 = plt.subplot(3,1,2) # (rows, columns, panel number)
plt.title("首都圏１都３県の感染速度（前週同曜日差の推移）({}時点)".format(latest_date_str))
target_p = ["東京都","神奈川県","埼玉県","千葉県"]
#stack_values = []
for p in target_p:
    values = chart_data.loc[chart_data["都道府県名"]==p]["新規感染者数先週差"]
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("陽性者数前週同曜日差(人)")


ax1 = plt.subplot(3,1,3) # (rows, columns, panel number)
plt.title("関西圏２府３県の感染速度（前週同曜日差の推移）({}時点)".format(latest_date_str))
target_p = ["大阪府","兵庫県","奈良県","京都府","和歌山県"]
for p in target_p:
    values = chart_data.loc[chart_data["都道府県名"]==p]["新規感染者数先週差"]
    plt.plot_date(dates,values,label=p, fmt="-", lw=1)
plt.plot([dates.min(),dates.max()],[0,0],lw=1,color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("陽性者数前週同曜日差(人)")


fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

<IPython.core.display.Javascript object>

Text(0, 0, '※新型コロナ関連の情報提供:NHK ／\u3000※人口の情報提供:e-Stat')

In [33]:
fig.savefig("out/covid-nhk-domestic-velocity-daily2.png")

/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 38525 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 24615 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 32773 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 25968 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 21069 missing from current font.
  

In [34]:
fig = plt.figure(figsize=(9.5,7))
plt.title("主要都道府県の１０万人当たり感染者数({}時点・全期間)".format(latest_date_str))
dates = pref["日付"].unique()
for p in prefs:
    if (p != "東京都") :
        continue
    values = pref.loc[pref["都道府県名"]==p]["10万人あたり感染者数"]
    plt.plot_date(dates,values,label=p, fmt=prefs[p][0], lw=prefs[p][1])
    if prefs[p][2] == "":
        text_date = dates.max()
    else:
        text_date = pd.to_datetime(prefs[p][2])
    text_y = pref.loc[(pref["都道府県名"]==p) & (pref["日付"]==text_date)]["10万人あたり感染者数"]
    plt.text(text_date,text_y,round(text_y.iloc[-1],2),fontsize=8,va="center")
    plt.plot([dates.min(),dates.max()],[text_y,text_y],"-",color="black")
plt.grid()
plt.legend(bbox_to_anchor=(0, 1),loc='upper left', fontsize=9)
plt.ylabel("人数（人口１０万人あたり）")
fig.text(0,0,"※新型コロナ関連の情報提供:NHK ／　※人口の情報提供:e-Stat",fontsize=9)

plt.tight_layout()

<IPython.core.display.Javascript object>

/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 20154 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 25968 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 65288 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 21475 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/masaru/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 65297 missing from current font.
  